In [1]:
from osgeo import gdal, gdalnumeric, ogr, osr
from PIL import Image, ImageDraw
import sys, os
from osgeo import gdal_array
import numpy as np
import matplotlib.pyplot as plt

In [3]:
# E:\Imágenes Satelitales\2017\226_86\1\LC08_L1TP_226086_20170116_20170311_01_T1_sr_band1.tif